In [1]:
import $file.common
import common._
import doobie._, doobie.implicits._
import cats.effect.IO

import $file.$     

import common._

import doobie._, doobie.implicits._

import cats.effect.IO

# Variation 3. Repositories

So, we want to abstract the persistence layer from our business logic. The common answer to this problem, at least in the Java community, is based on the [Repositories Pattern](https://martinfowler.com/eaaCatalog/repository.html). Basically, the persistence layer is isolated using conventional interfaces, as follows:

In [2]:
// Domain model

case class Country(code: String, name: String, capital: Option[Int])
case class City(id: Int, name: String, countryCode: String, population: Int)

// Repositories

trait CityRepo{
    def findCity(id: Int): City
    def findAllCities: List[City]
}

trait CountryRepo{
    def findCountry(code: String): Country
    def findAllCountries: List[Country]
}

trait WorldRepo extends CityRepo with CountryRepo

defined class Country
defined class City
defined trait CityRepo
defined trait CountryRepo
defined trait WorldRepo

Our query is implemented almost exactly as we did in the pure in-memory solution, only that the world dependency does not represent the actual data, but an interface to it:

In [7]:
def largeCities(implicit W: WorldRepo): List[(String, String)] = 
    for {
        Country(_, name, Some(capitalId)) <- W.findAllCountries
        city = W.findCity(capitalId) 
        if city.population > 8000000
    } yield (city.name, name)

defined function largeCities

This interface can be implemented so that they access the real world database through doobie:

In [4]:
class DoobieWorld(xa: Transactor[IO]) extends WorldRepo{
    def findCity(id: Int): City = 
        sql"select id, name, countryCode, population from city where id = $id"
            .query[City].unique.transact(xa).unsafeRunSync

    def findCountry(code: String): Country = 
        sql"select code, name, capital from country where code = $code"
            .query[Country].unique.transact(xa).unsafeRunSync
    
    def findAllCountries: List[Country] = 
        sql"select code, name, capital from country"
            .query[Country].to[List].transact(xa).unsafeRunSync
    
    def findAllCities: List[City] = 
        sql"select id, name, countryCode, population from city"
            .query[City].to[List].transact(xa).unsafeRunSync
}

object DoobieWorld extends DoobieWorld(xa)

defined class DoobieWorld
defined object DoobieWorld

So that, we can query the actual database by injecting this dependency to our generic program:

In [5]:
largeCities(DoobieWorld).timed(1).millis

7778 millis


res4: List[(String, String)] = List(
  ("Jakarta", "Indonesia"),
  ("Seoul", "South Korea"),
  ("Ciudad de M\u00e9xico", "Mexico"),
  ("Moscow", "Russian Federation")
)

Which other instances of these Repository APIs we would like to have? Well, not exactly proper instances, but we may mock the APIs in order to unit test our business logic. So, great! We got idiomatic code, modularity (we may decompose the `largeCities` query in smaller query fragments), and unit testing. And, since we are programming against an abstract interface, we may later add new instances without breaking our code, in case that we eventually change our persistence layer, right?

### Asynchronous Repositories

Well, not exactly. The APIs we just wrote seem to be purely generic, but they unadvertently commit to a particular class of implementations, namely __synchronous__ ones. Indeed, there is no asynchronous instance of the previous APIs. So, if we want to benefit from accessing the database asynchronously, we have to change our API, and ... all the business logic that builds upon it. 

This asynchronous version of the repository APIs may look like follows:

In [6]:
trait CityRepoAsync{
    def findCity(id: Int): IO[City]
}

trait CountryRepoAsync{
    def findCountry(code: String): IO[Country]
    def findAllCountries: IO[List[Country]]
}

trait WorldRepoAsync extends CityRepoAsync with CountryRepoAsync

defined trait CityRepoAsync
defined trait CountryRepoAsync
defined trait WorldRepoAsync

Admittedly, we are committing ourselves to a particular IO monad, namely `cats.effect.IO`, but this doesn't matter, right? Well, chances are there that we may need to implement our repositories using libraries based on alternative IO monads (`monix.Task`, `scalaz.Task`, `scala.concurrent.Future`, `ZIO`, etc.). That wouldn't be a stopper, but it would definitely require adaptor. Similarly, unit testing, which is far from being asynchronous, would have also to deal with IO. All this makes our code more complex than needed, and points towards a huge leak into our repository abstractions. Can we do it better? Enter [MTL-based repositories](Variation4.MTL.ipynb)!